# Part d: regression on 1Dim ising modellusing a multilayer perceptron (neural network)

In [1]:
import numpy as np
import scipy.sparse as sp
np.random.seed(12)

import warnings
#Comment this to turn on warnings
warnings.filterwarnings('ignore')

from model_comparison import model_comparison
from resample import resample
import algorithms
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns


from annCopy import NeuralNetMLP
from ann import NeuralNetMLPdeep
from utils import train_test_split
%matplotlib inline

In [2]:
### define Ising model aprams
# system size
L=40

# create 10000 random Ising states
states=np.random.choice([-1, 1], size=(10000,L))

def ising_energies(states,L):
    """
    This function calculates the energies of the states in the nn Ising Hamiltonian
    """
    J=np.zeros((L,L),)
    for i in range(L):
        J[i,(i+1)%L]-=1.0
    # compute energies
    E = np.einsum('...i,ij,...j->...',states,J,states)

    return E
# calculate Ising energies
energies=ising_energies(states,L)

"""
Comparing to project 1 - x = states, z = energies... 
Currently no y since we are in one dimension. 

"""

'\nComparing to project 1 - x = states, z = energies... \nCurrently no y since we are in one dimension. \n\n'

In [3]:
# reshape Ising states into RL samples: S_iS_j --> X_p
states=np.einsum('...i,...j->...ij', states, states)
shape=states.shape
states=states.reshape((shape[0],shape[1]*shape[2])) 
# build final data set
Data=[states,energies]

# define number of samples
n_samples=600
# define train and test data sets


In [4]:
X=Data[0]
Y=Data[1]

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, split_size=0.5)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((5000, 1600), (5000,), (5000, 1600), (5000,))

In [6]:
"""
ann = NeuralNetMLPdeep(l2 = 0.0001, 
                   batch_size=10, 
                   epochs=50, 
                   n_hidden=30,
                   eta = 0.01,
                    tpe = "regression")

ann.fit(X_train, y_train, X_valid, y_valid)
ann.predict(X_valid)
# returns a list of the mean mse score for different epochs or batches
metric.append(ann.eval_["valid_preform"])
print(" for lmd "+str(l) + " and eta: " + str(e) + "   performance is " + str(np.mean(ann.eval_["valid_preform"])))
"""

'\nann = NeuralNetMLPdeep(l2 = 0.0001, \n                   batch_size=10, \n                   epochs=50, \n                   n_hidden=30,\n                   eta = 0.01,\n                    tpe = "regression")\n\nann.fit(X_train, y_train, X_valid, y_valid)\nann.predict(X_valid)\n# returns a list of the mean mse score for different epochs or batches\nmetric.append(ann.eval_["valid_preform"])\nprint(" for lmd "+str(l) + " and eta: " + str(e) + "   performance is " + str(np.mean(ann.eval_["valid_preform"])))\n'

# Two layer MLP (Neural network )

In [9]:
# Experimental setup
metric = []

eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

for e in eta:
    for l in lmd:
        ann = NeuralNetMLP(n_hidden=30, 
                           l2=0.4,
                           epochs=100, 
                           eta=0.001, 
                           shuffle=True,
                            batch_size=10, 
                           seed=None, 
                           alpha=0.01, 
                           activation='elu', 
                           tpe = "logistic")
            
        ann.fit(X_train, y_train, X_valid, y_valid)
        ann.predict(X_valid)
        # returns a list of the mean mse score for different epochs or batches
        metric.append(ann.eval_["valid_preform"])
        print(" for lmd "+str(l) + " and eta: " + str(e) + "   performance is " + str(np.mean(ann.eval_["valid_preform"])))

AttributeError: 'NeuralNetMLP' object has no attribute 'tde'

In [ ]:
av = np.average(metric, axis = 1)

In [ ]:
av

In [ ]:
m = np.array(av).reshape((len(eta), len(lmd)))

plt.figure(figsize=(12,6))
ax = sns.heatmap(m, annot = True, xticklabels=lmd, yticklabels=eta)
ax.set_title("Regression using sigmoid function", fontsize = 20)
ax.set_xlabel("Lambda value", fontsize = 15)
ax.set_ylabel("Eta value", fontsize = 15) #
plt.savefig("./results/figures/regression_ridge_nn_sigmoid.png")

In [ ]:
# Experimental setup
metric = []

eta = [0.0001, 0.001, 0.01, 0.1]
lmd = [0.0001, 0.001, 0.01, 0.1, 1.0, 10]

for e in eta:
    for l in lmd:
        ann = NeuralNetMLP(l2 = l, 
                           batch_size=10, 
                           epochs=100, 
                           n_hidden=30,
                           eta = e,
                           tpe = "regression")

        ann.fit(X_train, y_train, X_valid, y_valid)
        ann.predict(X_valid)
        # returns a list of the mean mse score for different epochs or batches
        metric.append(ann.eval_["valid_preform"])
        print(" for lmd "+str(l) + " and eta: " + str(e) + "   performance is " + str(np.mean(ann.eval_["valid_preform"])))

# Comparing to scikitlearn MLPregression

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(30, ), activation = 'logistic', solver = "sgd", alpha = 0.0001, batch_size =10, learning_rate_init=0.01)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_valid)

np.sum(np.square(y_pred - y_valid))/len(y_pred)

In [ ]:
y_pred